In [44]:
using Random
using NBInclude
@nbinclude("dcj_algo.ipynb")
@nbinclude("testing_diameter.ipynb")
@nbinclude("randommap.ipynb")

In [46]:
# random generation of genomes; distribution of #dcj operations 

function generate_genomes_with_1dup(n::Int)
    genes = first_n_letters(n)
    rand_dup_gene = genes[rand(1:n)]
    push!(genes, rand_dup_gene)

    target = join(genes)

    linear = rand(Bool)
    rand_idx = rand(1:n)
    src = generate_genome_str(n)
    if linear 
        rand_dup_gene = "." * rand_dup_gene * "."
    end 
    src = src[1:rand_idx-1] * string(rand_dup_gene) * src[rand_idx:end]

    print(src, " --> ", target)
    return src, target
end 

n = 5  # num genes
num_maps = 2

src, target = generate_genomes_with_1dup(n)

dcj_dist_list = randommap(src, target, num_maps, "info")

# calculate_distance(src, target, "none")

# one or two duplicated genes; just sample different deduplication permutations -- measure variability in dcj distance 
    # min and max 
    # probability distribution 
    # want to show there's a huge variability betewen what map you choose 


    

Ecbda,.c. --> abcdecSRC Ecbda,.c. --> TARGET abcdec

target
dup genes --> multiplicity
OrderedDict('c' => 2)

target map
[1]

source
dup genes --> multiplicity
OrderedDict('c' => 2, '.' => 2)

source maps
[1, 2]
[2, 1]

deduplication
deduplicated target abcdet



LoadError: KeyError: key '.' not found